In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import math
import torch.nn as nn
import math
from layers import BBB_Linear, BBB_Conv2d
from layers import BBB_LRT_Linear, BBB_LRT_Conv2d
from layers import FlattenLayer, ModuleWrapper



In [7]:
import torch
print("uuu")
import torch.nn as nn
print("saaaaaah")
import torch.optim as optim
print("pffff")
from tqdm import tqdm

print("tutu")

# Assurez-vous d'importer les modules BBB nécessaires
from PyBCNN.layers.BBB import BBBLinear as BBB_Linear, BBBConv as BBB_Conv
from PyBCNN.layers.BBB_LRT import BBBLinear as BBB_Linear_LRT, BBBConv as BBB_Conv_LRT
from PyBCNN.layers.misc import FlattenLayer, ModuleWrapper
from PyBCNN.models.BayesianModels.BayesianAlexNet import BBBAlexNet
from torch.utils.data import random_split




#################################################################################################


from torchvision.models import alexnet

uuu
saaaaaah
pffff
tutu


In [1]:

# Vérification de l'utilisation du gpu
!nvidia-smi 

# Optimisation Pytorch
torch.backends.cudnn.benchmark = True  # Accélère les convolutions
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Utilisation de :", device)

Sat Nov 15 11:37:01 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.30                 Driver Version: 546.30       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8              14W /  95W |   2975MiB /  6144MiB |     29%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

NameError: name 'torch' is not defined

Implementation de AlexNet

Imporation des libraries

Creation de l'architecture ##########################

-Feature Extractor: La première couche de convolution prend une image à 3 canaux et passe un kernel de taille 11x11 avec un pas de 4 réduit. Les fonction d'activation sont des ReLu (conserve les y positives et renvoi 0 pour les y négatifs) Ensuite, un maxpooling est appliqué pour réduire la dimension des images et concerver les informations importantes

-Classifier: Ensemble de percptron multi-couches avec des fonction d'activation ReLu Les features (les x) sont projetées dans un espace de grande dimension ici de taille 4096. La dernière couches est la sortie du reseau

-Forward: Calcul du lien entre les entrées et les sorties (f(x)) ou on défini le passage des entrées dans le Feature extractor, les features sont transformées en vecteur (fletten) puis les vecteurs de features sont passées dans les PMC.

In [3]:

# 1. Charger AlexNet pré-entraîné
pretrained_alexnet = alexnet(pretrained=True)

# 2. Créer BBBAlexNet
priors = {
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0.0, 0.1),     # (mean, std)
    'posterior_rho_initial': (-3.0, 0.1)    # souvent un petit std
}

model = BBBAlexNet(
    outputs=2,
    inputs=3,
    priors=priors,
    layer_type='bbb',
    activation_type='relu'  # Utilisez 'relu' pour matcher AlexNet
)



C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:

# 3. Transférer les poids vers les moyennes (mu) des distributions bayésiennes
def transfer_weights_to_bayesian(bayesian_model, pretrained_model):
    """
    Transfère les poids d'un modèle classique vers les moyennes (mu) 
    des distributions bayésiennes
    """
    # Mapping des couches conv
    conv_mapping = {
        'conv1': 0,  # pretrained.features[0] -> bayesian.conv1
        'conv2': 3,  # pretrained.features[3] -> bayesian.conv2
        'conv3': 6,  # pretrained.features[6] -> bayesian.conv3
        'conv4': 8,  # pretrained.features[8] -> bayesian.conv4
        'conv5': 10, # pretrained.features[10] -> bayesian.conv5
    }
    
    # Transférer les couches de convolution
    for bay_name, pre_idx in conv_mapping.items():
        bay_layer = getattr(bayesian_model, bay_name)
        pre_layer = pretrained_model.features[pre_idx]
        
        # Copier weight vers weight_mu
        bay_layer.W_mu.data.copy_(pre_layer.weight.data)
        if pre_layer.bias is not None:
            bay_layer.bias_mu.data.copy_(pre_layer.bias.data)
        
        print(f"✅ Transféré {bay_name} depuis features[{pre_idx}]")
    
    # Transférer la couche classifier (adapter selon votre nombre de classes)
    # Note: AlexNet a 1000 classes, vous en avez 2
    # On ne transfère donc PAS la dernière couche, juste les features
    
    print("✅ Transfert terminé ! Les moyennes bayésiennes sont initialisées avec AlexNet pré-entraîné")

# Appliquer le transfert
transfer_weights_to_bayesian(model, pretrained_alexnet)

# Maintenant model a ses moyennes initialisées avec les poids d'AlexNet
# Les sigmas restent à leur valeur initiale (petite variance)

################ J'ai du modifier 
#        self.conv5 = BBBConv2d(256, 128, 3, padding=1, bias=True, priors=self.priors)
# pour 
#        self.conv5 = BBBConv2d(256, 256, 3, padding=1, bias=True, priors=self.priors)


✅ Transféré conv1 depuis features[0]
✅ Transféré conv2 depuis features[3]
✅ Transféré conv3 depuis features[6]
✅ Transféré conv4 depuis features[8]
✅ Transféré conv5 depuis features[10]
✅ Transfert terminé ! Les moyennes bayésiennes sont initialisées avec AlexNet pré-entraîné


In [5]:
# === FONCTION LOSS ===
criterion = nn.CrossEntropyLoss()

# === OPTIMIZER ===
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

# === SCHEDULER ===
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [8]:
# === 1. Chargement des images en Tensor (sans normalisation)
root = r"DATA/data_test_bug_ant_bee"

temp_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor()
])

temp_dataset = datasets.ImageFolder(root=root, transform=temp_transform)
temp_loader = DataLoader(temp_dataset, batch_size=64, shuffle=False)

# === 2. Calcul des moyennes et écarts-types
mean = 0.0
std = 0.0
total_images = 0

for images, _ in temp_loader:
    batch_samples = images.size(0)  # nombre d'images dans le batch
    images = images.view(batch_samples, images.size(1), -1)  # (batch, 3, H*W)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_images += batch_samples

mean /= total_images
std /= total_images

print("✅ Mean :", mean)
print("✅ Std  :", std)

# === 3. Transformation finale avec tes valeurs calculées
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist()),
])

# === 4. Chargement du dataset complet avec normalisation adaptée
dataset = datasets.ImageFolder(root=root, transform=transform)

# === 5. Division en train/test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# === 6. DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size=128, shuffle=True,
    num_workers=0, pin_memory=True, persistent_workers=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=0, pin_memory=True, persistent_workers=False
)

print(f"Nombre d'images total : {len(dataset)}")
print(f"Train : {len(train_dataset)}, Test : {test_size}")
print(f"Classes : {dataset.classes}")

✅ Mean : tensor([0.4425, 0.5082, 0.2139])
✅ Std  : tensor([0.1524, 0.1685, 0.0976])
Nombre d'images total : 2132
Train : 1705, Test : 427
Classes : ['ant', 'bee']


In [9]:

# === BOUCLE D'ENTRAÎNEMENT BAYÉSIENNE ===
num_epochs = 10
num_batches = len(train_loader)  # Important pour normaliser le KL loss

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_nll = 0.0
    running_kl = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        try:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        except Exception as e:
            print(f"Error loading data for epoch {epoch+1}: {e}")
            continue

        optimizer.zero_grad(set_to_none=True)
        
        # Forward pass
        outputs = model(images)
        
        # Negative Log-Likelihood (NLL) loss
        nll_loss = criterion(outputs, labels)
        
        # KL Divergence loss (terme de régularisation bayésien)
        kl_loss = model.kl_loss()  # Méthode fournie par ModuleWrapper
        
        # Loss totale = NLL + KL/nombre_de_batches
        # Le facteur 1/num_batches permet de pondérer correctement le KL
        loss = nll_loss + (kl_loss / num_batches)
        
        # Backward et mise à jour
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_nll += nll_loss.item()
        running_kl += kl_loss.item()

    scheduler.step()

    # Affichage des losses
    avg_loss = running_loss / len(train_loader)
    avg_nll = running_nll / len(train_loader)
    avg_kl = running_kl / len(train_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f} | NLL: {avg_nll:.4f} | KL: {avg_kl:.4f}")

Epoch 1/10:   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1/10:   7%|▋         | 1/14 [00:00<00:06,  2.12it/s]

Error loading data for epoch 1: name 'device' is not defined


Epoch 1/10:  14%|█▍        | 2/14 [00:00<00:05,  2.22it/s]

Error loading data for epoch 1: name 'device' is not defined


Epoch 1/10:  21%|██▏       | 3/14 [00:01<00:05,  1.96it/s]

...

Epoch 10/10:  79%|███████▊  | 11/14 [00:04<00:01,  2.08it/s]

Error loading data for epoch 10: name 'device' is not defined


Epoch 10/10:  86%|████████▌ | 12/14 [00:05<00:00,  2.15it/s]

Error loading data for epoch 10: name 'device' is not defined


Error loading data for epoch 10: name 'device' is not defined
Error loading data for epoch 10: name 'device' is not defined
Epoch [10/10] - Loss: 0.0000 | NLL: 0.0000 | KL: 0.0000


In [ ]:

# === VALIDATION BAYÉSIENNE ===
model.eval()
total, correct = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        # Pour un réseau bayésien, vous pouvez faire plusieurs prédictions (Monte Carlo)
        # et moyenner les résultats pour avoir une meilleure estimation
        num_samples = 10  # Nombre d'échantillonnages Monte Carlo
        predictions = []
        
        for _ in range(num_samples):
            outputs = model(images)
            predictions.append(torch.softmax(outputs, dim=1))
        
        # Moyenne des prédictions
        avg_predictions = torch.stack(predictions).mean(0)
        _, predicted = avg_predictions.max(1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')